### Selenium으로 인스타그램 크롤링 수도코드
필요한 정보: 5000개의 게시글. 게시글 안에는 게시글 내용, 게시자 계정, 날짜, 댓글단 계정, 댓글 내용

In [2]:
'''
# 필요한 라이브러리 import
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# 키워드에 대한 검색 결과를 반환하는 함수
def instagram_crawler(keyword):
    # 데이터
    data = []
    
    # 페이지 불러오기
    driver = webdriver.Chrome('./chromedriver/chromedriver.exe')
    driver.get('https://www.instagram.com/explore/tags/{$keyword}/?hl=ko'%keyword)
    sleep(2)
    
    # 다음 페이지 클릭이 아니라 스크롤을 내려야 함
    actions = ActionChains(driver)
    
    # 페이지 로드
    while len(data) <5001:
        content = BeautifulSoup(driver.page_source, 'html.parser')
        list_items = content.findAll('div',{'class':'찾고자 하는 클래스'}) #검색할 때마다 태그가 바뀌고 규칙X
        imgs = content.findAll('img', {'class':'찾고자 하는 클래스'})
        dates = content.findAll('time')
        ids = content.findAll('a', {'class':'찾고자 하는 클래스'})
        cmts = content.findAll('datetime')
        
        for _ in imgs:
            a = Article(img, date, id)
        if cmts:
            for c in cmts:
                a.comments(cmt_id, cmt)
        data.append(a)
        
        # 다음 페이지 클릭이 아니라 스크롤을 내려야 함
        actions.send_keys(Keys.SPACE).perform()
        sleep(1)

# 데이터 객체
class Article:
    dic = {}
    
    def __init__(self, img, date, id):
        self.img = img
        self.date = date
        self.id = id
    
    def comments(self, cmt_id, cmt):
        self.dic[cmt_id] = cmt
        
'''
print("")

### 영화 사이트 크롤링
https://movie.daum.net/boxoffice/yearly?year

In [5]:
'''
수행:
2019년, 2018년 2개 년도의 [제목, 네티즌 평점, 평론가 평점, 누적 관객수]
19년도에 누락된 데이터가 있고, 일부 항목은 클릭해서 들어가야 알 수 있음
누락 데이터는 NA나 없음 등으로 적절하게 처리
100개 영화에 대한 정보를 csv / json 형식으로 만들기

목표:
모듈화(각 함수의 역할이 명확함)
End-to-end (한 번의 실행으로 모든 작업이 이뤄짐) 

'''
print("")

In [41]:
'''
누적 관객수 구하는 방법
웹 페이지를 모두 받은 후 '누적관객수'를 검색해보면 해당 데이터는 웹 페이지 안에서 ajax 호출로 받아온 것이다.

        // 누적관객수
		jQuery.get("/moviedb/main/totalAudience.json", {movieId:109924}, function(data) {
			if(data != null && data.res == "OK") {
				$("#totalAudience").html(data.totalAudience + "명");
			} else {
				var htmlcode = "<span class='invalid_one'>(자료없음)</span>";
				$("#totalAudience").html(htmlcode);
			}
		});
        
ajax를 사용한 데이터를 불러오는 방법 (크롬 기준)
1. 개발자 도구 진입
2. Network 탭 클릭
3. F5 누르고 원하는 데이터를 목록에서 찾기
4. Open in new tab
5. url 찾아서 requests로 받아오기
'''
print("")

In [42]:
import pandas as pd
import re
import time
from time import sleep
from bs4 import BeautifulSoup as bs
import requests
import random
from random import randint
import json

In [79]:
def daum_movie_crawler(year):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    main_url = 'https://movie.daum.net/boxoffice/yearly?year=' + str(year)
    
    # Initialize 
    databox = []
    cnt= 0 
    
    response = requests.get(main_url,headers=headers)
    body_table = bs(response.text,'lxml')
    poster_table = body_table.find_all('div',class_='wrap_movie')
    
    for poster in poster_table:
        data = []
        try:
            title = poster.find('a', class_='name_movie #title')
            data.append(title.text)
            
        except Exception as e :
            data.append('NA')
            print(e,'Error at {}'.format(title.text))
            
        try:
            netizen_grade_table = poster.find("span", {"class":"wrap_grade grade_netizen"})
            netizen_grade = netizen_grade_table.find("span", {"class": re.compile("num_grade")})
            data.append(netizen_grade.text)
            
        except Exception as e :
            data.append('NA')
            print(e,'Error at {}'.format(title.text))
        
        try:
            critic_grade_table = poster.find("span",{"class":"wrap_grade grade_critic"})
            critic_grade = critic_grade_table.find("span", {"class": re.compile("num_grade")})
            data.append(critic_grade.text) 
            
        except Exception as e :
            data.append('NA')
            print(e,'Error at {}'.format(title.text))
            
        try:
            mvid = title.get('href').split('=')[1]
            url = "https://movie.daum.net/moviedb/main/totalAudience.json?movieId=" + mvid
            js = requests.get(url,headers=headers).text
            dic = json.loads(js)
            data.append(dic['totalAudience'])
                                                      
        except Exception as e :
            data.append('NA')
            print(e,'Error at {}'.format(title.text))
            
        databox.append(data)

    return databox

In [84]:
want_year = [2018, 2019]
for year in want_year:
    d = pd.DataFrame(daum_movie_crawler(year))
    d.columns = ['제목', '네티즌 평점', '전문가 평점', '누적 관객수']
    d.to_csv('{}년 영화 정보.csv'.format(year))

'NoneType' object has no attribute 'find' Error at 데드풀 2
'totalAudience' Error at 데드풀 2
'NoneType' object has no attribute 'find' Error at 극한직업
'NoneType' object has no attribute 'find' Error at 어벤져스: 엔드게임
'NoneType' object has no attribute 'find' Error at 알라딘
'NoneType' object has no attribute 'find' Error at 기생충
'NoneType' object has no attribute 'find' Error at 스파이더맨: 파 프롬 홈
'NoneType' object has no attribute 'find' Error at 엑시트
'NoneType' object has no attribute 'find' Error at 캡틴 마블
'NoneType' object has no attribute 'find' Error at 라이온 킹
'NoneType' object has no attribute 'find' Error at 봉오동 전투
'NoneType' object has no attribute 'find' Error at 토이 스토리 4
'NoneType' object has no attribute 'find' Error at 돈
'NoneType' object has no attribute 'find' Error at 악인전
'NoneType' object has no attribute 'find' Error at 증인
'NoneType' object has no attribute 'find' Error at 분노의 질주: 홉스&쇼
'NoneType' object has no attribute 'find' Error at 사바하
'NoneType' object has no attribute 'find' Error at 

In [85]:
mv2018 = pd.read_csv("2018년 영화 정보.csv", index_col=0)
mv2019 = pd.read_csv("2018년 영화 정보.csv", index_col=0)

In [86]:
mv2018

,제목,네티즌 평점,전문가 평점,누적 관객수
0,신과함께-인과 연,6,6.0,"12,276,115"
1,어벤져스: 인피니티 워,7,7.0,"11,212,710"
2,보헤미안 랩소디,9,6.0,"9,948,386"
3,미션 임파서블: 폴아웃,8,7.0,"6,584,919"
4,신과함께-죄와 벌,7,6.0,"14,411,675"
5,쥬라기 월드: 폴른 킹덤,7,6.0,"5,661,128"
6,앤트맨과 와스프,7,6.0,"5,448,134"
7,안시성,7,6.0,"5,441,020"
8,블랙 팬서,6,6.0,"5,399,327"
9,완벽한 타인,7,6.0,"5,294,154"


In [87]:
mv2019

,제목,네티즌 평점,전문가 평점,누적 관객수
0,신과함께-인과 연,6,6.0,"12,276,115"
1,어벤져스: 인피니티 워,7,7.0,"11,212,710"
2,보헤미안 랩소디,9,6.0,"9,948,386"
3,미션 임파서블: 폴아웃,8,7.0,"6,584,919"
4,신과함께-죄와 벌,7,6.0,"14,411,675"
5,쥬라기 월드: 폴른 킹덤,7,6.0,"5,661,128"
6,앤트맨과 와스프,7,6.0,"5,448,134"
7,안시성,7,6.0,"5,441,020"
8,블랙 팬서,6,6.0,"5,399,327"
9,완벽한 타인,7,6.0,"5,294,154"
